# MobileNet V2 with Transfer Learning

## Imports and Dataset

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [3]:
! git clone https://github.com/Afeefaa/AWS

fatal: destination path 'AWS' already exists and is not an empty directory.


In [4]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Dropout, SpatialDropout2D, AveragePooling2D, BatchNormalization
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, show, imshow
import cv2
from keras.preprocessing.image import ImageDataGenerator

## Data Augmentation and Lazy Loading

In [5]:
os.chdir('AWS')
batchsize = 35
# Data Augmentation

train_datagen = ImageDataGenerator(
   	shear_range=0.1,	height_shift_range=0.2,
        zoom_range=0.2,     horizontal_flip=True,
        vertical_flip=True, width_shift_range=0.2)

# Lazy Loading

train_generator = train_datagen.flow_from_directory(
        'train',           target_size=(256, 256),
        batch_size=batchsize,      class_mode='categorical')

Found 1995 images belonging to 6 classes.


## Validation Set Pre-Processing

In [6]:
import pickle
try:
	x_val=pickle.load(open('x_val.bin','rb'))
	y_val=pickle.load(open('y_val.bin','rb'))

except:
	os.chdir('predb')

	y_val = np.array([]).reshape((0,6))
	x_val = np.array([]).reshape((0,256,256,3))

	def encode_class(x):
		l = [0 for i in range(6)]
		l[x]=1
		return np.array(l).reshape((1,6))

	def load_data(name):
		global x_val
		global y_val

		x=5

		global image
		image = plt.imread(name)
		image = cv2.resize(image,(256,256))


		x_val = np.append(x_val, image.reshape((1,256,256,3)),axis=0)

		if 'cardboard' in name:
			x=0
		elif 'glass' in name:
			x=1
		elif 'metal' in name:
			x=2
		elif 'paper' in name:
			x=3
		elif 'plastic' in name:
			x=4

		y_val = np.append(y_val, encode_class(x),axis=0)

	for i in os.listdir()[1:]:
		load_data(i)

	print(x_val.shape, '\t', y_val[0])
	os.chdir("..")

	with open("x_val_256.bin", 'wb') as f:
		pickle.dump(x_val,f)

	with open("y_val_256.bin", 'wb') as f:
		pickle.dump(y_val,f)

In [7]:
IMG_SIZE = (160, 160)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=True,
                                               weights='imagenet')

In [8]:
base_model.summary()
base_model.trainable = False

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][

In [9]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    ### START CODE HERE
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    ### END CODE HERE

    return data_augmentation

In [13]:
def alpaca_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''

    input_shape = image_shape + (3,)

    ### START CODE HERE

    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False, # <== Important!!!!
                                                   weights='imagenet') # From imageNet

    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape)

    # apply data augmentation to the inputs
    x = data_augmenter()(inputs)

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False)

    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)

    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(6)(x)

    ### END CODE HERE

    model = tf.keras.Model(inputs, outputs)

    return model

In [14]:
model2 = alpaca_model((256, 256))

In [15]:
base_learning_rate = 0.001
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
# Checkpoint

checkpoint = keras.callbacks.ModelCheckpoint('ckpt_og.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Early Stopping

stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

try:
    model2.load_weights('ckpt_og.keras')
except:
    print("No Pre-existing Checkpoint, creating new one")

No Pre-existing Checkpoint, creating new one


In [ ]:
initial_epochs = 5
history = model2.fit(train_generator, steps_per_epoch=1995//batchsize,
        epochs=20, validation_data=(x_val,y_val), callbacks=[checkpoint, stop])

Epoch 1/20
57/57 [==============================] - ETA: 0s - loss: 0.3812 - accuracy: 0.4867
Epoch 1: val_accuracy improved from -inf to 0.73956, saving model to ckpt_og.keras
57/57 [==============================] - 40s 521ms/step - loss: 0.3812 - accuracy: 0.4867 - val_loss: 0.2680 - val_accuracy: 0.7396
Epoch 2/20
57/57 [==============================] - ETA: 0s - loss: 0.2472 - accuracy: 0.7268
Epoch 2: val_accuracy improved from 0.73956 to 0.79115, saving model to ckpt_og.keras
57/57 [==============================] - 29s 510ms/step - loss: 0.2472 - accuracy: 0.7268 - val_loss: 0.2231 - val_accuracy: 0.7912
Epoch 3/20
57/57 [==============================] - ETA: 0s - loss: 0.2095 - accuracy: 0.7704
Epoch 3: val_accuracy improved from 0.79115 to 0.80098, saving model to ckpt_og.keras
57/57 [==============================] - 33s 575ms/step - loss: 0.2095 - accuracy: 0.7704 - val_loss: 0.2019 - val_accuracy: 0.8010
Epoch 4/20
57/57 [==============================] - ETA: 0s - loss:

In [ ]:
base_model = model2.layers[4]
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 120

### START CODE HERE

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Define a BinaryCrossentropy loss function. Use from_logits=True
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Define an Adam optimizer with a learning rate of 0.1 * base_learning_rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1*base_learning_rate)
# Use accuracy as evaluation metric
metrics= ['accuracy']

### END CODE HERE

model2.compile(loss=loss_function,
              optimizer = optimizer,
              metrics=metrics)

In [ ]:
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs
try:
    model2.load_weights('ckpt_og.keras')
    print("Checkpoint Loaded")
except:
    print("No Pre-existing Checkpoint, creating new one")

history_fine = model2.fit(train_generator, steps_per_epoch=1995//batchsize,
        epochs=20, validation_data=(x_val,y_val), callbacks=[checkpoint, stop])

In [ ]:
model2.compile(loss=loss_function,
              optimizer = optimizer,
              metrics=metrics)